In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
import numpy as np

%matplotlib inline
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('..//data/cleaned_pets.csv')

C:\Users\Max-X1\AppData\Local\Temp\ipykernel_2076\3667737777.py:1: DtypeWarning: Columns (17,18,22,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('..//data/cleaned_pets.csv')


In [4]:
df.shape

(34009, 42)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34009 entries, 0 to 34008
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          34008 non-null  float64
 1   organization_id             34008 non-null  object 
 2   url                         34008 non-null  object 
 3   type                        34008 non-null  object 
 4   species                     34008 non-null  object 
 5   age                         34008 non-null  object 
 6   gender                      34008 non-null  object 
 7   size                        34008 non-null  object 
 8   coat                        13599 non-null  object 
 9   tags                        12221 non-null  object 
 10  name                        34008 non-null  object 
 11  description                 19495 non-null  object 
 12  status                      34008 non-null  object 
 13  status_changed_at           340

In [6]:
# Convert to datetime
df['published_at'] = pd.to_datetime(df['published_at'])
df['status_changed_at'] = pd.to_datetime(df['status_changed_at'])
df['published_date'] = pd.to_datetime(df['published_date'])
df['status_change_date'] = pd.to_datetime(df['status_change_date'])

In [7]:
# Define function to switch dtypes to string
def string_clean(column):
    x = df[column].astype('string')
    return x

# Switch objects to strings
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = string_clean(column)

In [8]:
#df.info()

In [9]:
# Create adopted and adoptable dataframes
adopted = df[(df['year_status_change'] == 2023) & (df['status'] == 'adopted')]
adoptable = df[(df['year_status_change'] == 2023) & (df['status'] == 'adoptable')]

# All Animals Analysis

### Overall understanding of adoptions in TN

#### How many animals were placed for adoption in 2023?

In [9]:
# Subset data for 2023 
published = df[df['year_published'] == 2023]
published = published['id'].count()

print('Number of pets placed for adoption in 2023: ', published)

Number of pets placed for adoption in 2023:  34008


##### How animals many were adopted? 

In [10]:
adopted = df[(df['year_status_change'] == 2023) & (df['status'] == 'adopted')]
adopted = adopted['id'].count()

print('Number of pets adopted in 2023:', adopted)

Number of pets adopted in 2023: 29735


##### What % of animals were adopted? What % of animals were NOT adopted?

In [11]:
adopted_perc = round((adopted / published) * 100, 1)
print('% of pets adopted:', adopted_perc, '%')

not_adopted_perc = round(((published - adopted) / published) * 100, 1)
print('% of pets not adopted in 2023:', not_adopted_perc, '%')

% of pets adopted: 87.4 %
% of pets not adopted in 2023: 12.6 %


### Overview of types of animals & adoption rates

##### What is the distribution of animals on petfinder?

In [12]:
# Group ids by species and count
published_species = df.groupby('species')['id'].count().reset_index(name='count published').sort_values('count published', ascending=False)
published_species['percentage %'] = round((published_species['count published'] / df['id'].count()) * 100, 3)

published_species

,species,count published,percentage %
1,Dog,19676,57.857
0,Cat,14116,41.508
11,Rabbit,108,0.318
3,Guinea Pig,57,0.168
10,Pot Bellied,18,0.053
6,Horse,12,0.035
9,Pig,5,0.015
2,Goat,4,0.012
7,Miniature Horse,3,0.009
4,Hamster,2,0.006


##### Visualize

##### How many pets of each species were adopted in 2023? What is the distribution of species?

In [14]:
adopted_species = adopted.groupby('species')['id'].count().reset_index(name='count adopted').sort_values('count adopted', ascending=False)
adopted_species['percentage %'] = round((adopted_species['count adopted'] / adopted_species['count adopted'].sum()) * 100, 3)
adopted_species

AttributeError: 'numpy.int64' object has no attribute 'groupby'

#### Visualize

##### What is the adoption rate by species? 

In [15]:
# Outer join published species and adopted species
adoption_rate = pd.merge(published_species, adopted_species, on='species', how='outer')

# Fill NaN values with zero
adoption_rate = adoption_rate.fillna(0)

# Convert 'count adopted' column to integer
adoption_rate['count adopted'] = adoption_rate['count adopted'].astype(int)

# Calculate the adoption rate %
adoption_rate['adoption rate %'] = round((adoption_rate['count adopted'] / adoption_rate['count published']) * 100, 2)
adoption_rate.sort_values('adoption rate %', ascending=False)

NameError: name 'adopted_species' is not defined

##### What is the average adoption time (in days) per species? Which species has the longest adoption time?

In [ ]:
# Calculate average adoption time by species
avg_time = adopted.groupby('species')['adoption_time'].mean().reset_index(name='avg_adoption_time').sort_values('avg_adoption_time', ascending=False)

print('Answer: Rabbits:', round(avg_time['avg_adoption_time'].max(), 2), 'days')
avg_time

### Overview of age

##### What is the adoption rate by age? What age group has the highest and lowest adoption rate? 

Answer: Highest - Baby, Lowest - Seniors 

In [10]:
# Group id by age and count for published and adopted dfs
pub_age_count = df.groupby('age')['id'].count().reset_index(name='count published')
adopted_age_count = adopted.groupby('age')['id'].count().reset_index(name='count adopted')

# Merge two dfs together
age_adoption_rate = pd.merge(pub_age_count, adopted_age_count, on='age')

# Calculate adoption rate
age_adoption_rate['adoption rate %'] = round((age_adoption_rate['count adopted'] / age_adoption_rate['count published']) * 100, 2)

age_adoption_rate.sort_values('adoption rate %', ascending=False)

,age,count published,count adopted,adoption rate %
1,Baby,8565,7668,89.53
0,Adult,16477,14741,89.46
2,Senior,1479,1313,88.78
3,Young,7487,6013,80.31


# City Analysis

In [ ]:
# Group id by city and count for published and adopted dfs
pub_city_count = df.groupby('contact.address.city')['id'].count().reset_index(name='count published')
adopted_city_count = adopted.groupby('contact.address.city')['id'].count().reset_index(name='count adopted')

# Merge two dfs together
city_adoption_rate = pd.merge(pub_city_count, adopted_city_count, on='contact.address.city')

# Calculate adoption rate
city_adoption_rate['adoption rate %'] = round((city_adoption_rate['count adopted'] / city_adoption_rate['count published']) * 100, 2)

city_adoption_rate

#### Which cities (out of the top 10) had the most pets available for adoption? Which city in the top 10 had the highest adoption rate?

In [ ]:
top_10 = city_adoption_rate.sort_values('count published', ascending=False).head(10)
top_10

# Answer: Greenville had the highest adoption rate

# Dog Analysis

In [ ]:
dogs = df[df['species'] == 'Dog']
print(dogs.shape)
dogs.groupby('type')['id'].count()

#### What is the overall adoption rate for dogs?

#### How many full breeds vs mixed breeds were listed for adoption? What is the adoption rates for each type?

In [ ]:
full = dogs[dogs['breeds.mixed'] == False]
mixed = dogs[dogs['breeds.mixed'] == True]

print('Full Breed Dogs: ', full['id'].count())
print('Mixed Breed Dogs: ', mixed['id'].count())

# add adoption rate

#### What are the top 20 breeds listed for adoption? Are they full or mixed breeds?

In [ ]:
# Group dogs df by primary breed and mixed breed columns and count ids
top_20 = dogs.groupby(['breeds.primary', 'breeds.mixed'])['id'].count().reset_index(name='count').sort_values('count', ascending=False).head(20)

top_20

#### What are the top 20 full breeds and top 20 mixed breeds (include secondary breed)?

In [ ]:
# Top 20 full breeds

# Group ids by primary breed
top_20_fb = full.groupby('breeds.primary')['id'].count().reset_index(name='count').sort_values('count', ascending=False).head(21)

# Drop index 100 because it is not relevant
#top_20_fb = top_20_fb.drop(100)
top_20_fb